# Phonopy and ASE Tutorial

In this tutorial you will learn how to run an automated calculation of phonons by calculating the forces using ASE.

We will use the `aiida_phonopy.workflows.ase.PhonopyAseWorkChain`, which is based on `aiida_pythonjob`.
This workflow takes care of: 

* Pre-processing: generating the (supercell) structures with displacements on top of which computing forces (for frozen phonons)
* Gather all information in `PhonopyData`, ready to be post-processed 
* (optional) Post-process: calculate phonon-related properties, such as phonon band structure and (P)DOS, thermal properties, and so on by using `PhonopyCalculation`.

In this tutorial we will make use of the silicon structure to give you an overall understanding of the usage.

Let's get started!

In [6]:
from local_module import load_temp_profile
from aiida.plugins import DataFactory, WorkflowFactory

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
load_temp_profile(
    name="ase-tutorial",
    add_computer=True,
    add_phonopy_code=True,
)

StructureData = DataFactory("core.structure")
PhonopyAseWorkChain = WorkflowFactory("phonopy.ase")

Let's define the alumin structure using the ASE module

In [7]:
from ase.build import bulk

atoms = bulk("Al", a=1.5456658) # Note: this is NOT the experimental lattice constant, but the good value for the force field used in the tutorial
structure = StructureData(ase=atoms)

```{code-block} python
:caption: |
:    If you have your own structure, e.g., 
:    in .cif or .xyz format, you can simply use the followig snippet

from ase.io import read

atoms = read("/path/to/file.cif") # here, any format supported by ASE
structure = StructureData(ase=atoms)
```

## Automated calculation via `PhonopyAseWorkChain`

We now want to choose an ASE `calculator` that we want to give to the workchain, so that it will compute with it all the forces on all the displaced structures. The calculation will be performed automatically by the `PhonopyAseWorkChain`, which will give us a `PhonopyData` as output that stores all the displacements and forces needed to compute phonons-related properties.

The calculator can be:
- a simple empirical forces field, like a Lenard-Jones potential (presented in this example)
- a DFT calculator (e.g., Quantum ESPRESSO, VASP, Abinit, and so on, interfaced by ASE)
- a pre-trained machine-learning potential (e.g., NequIP, Allegro, MACE, MatterSim, GAP, FLARE, and so on, interfaced with ASE)

These should be installed and able to run on your machine or on a remote cluster.

In [ ]:
from ase.calculators.lj import LennardJones
from aiida.orm import Dict, load_code
from aiida.engine import run_get_node

inputs = PhonopyAseWorkChain.get_populated_builder(
    structure=structure,
    calculator=LennardJones(),
    max_number_of_atoms=200,
    pythonjob_inputs={"computer": "local_direct"},
    phonopy_inputs={
        "code": load_code("phonopy@local_direct"),
        "parameters": Dict({"band":"auto"})
    },
)

results, node = run_get_node(PhonopyAseWorkChain, **inputs)

In [ ]:
results['output_phonopy']['phonon_bands'].show_mpl()

## Manual post-processing

You can still of course use the output `PhonopyData` to get a `Phonopy` instance, allowing you to directly post-process the data locally.

In [ ]:
ph = node.outputs.phonopy_data.get_phonopy_instance()
ph.produce_force_constants()
ph.auto_band_structure(plot=True)